In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("Users/adityasharma/Github Projects/Amazon/input/train.csv")

In [ ]:
import os
hf_token = os.getenv("HF_TOKEN")

In [ ]:
df.shape

(75000, 4)

In [ ]:
df.head()

,sample_id,catalog_content,image_link,price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49


In [ ]:
import re

def extract_value_unit_from_value_block(text):
    # Find all "Value: X" followed by "Unit: Y" pairs, use last occurrence
    pattern = r'Value:\s*([0-9,]+(?:\.\d+)?)\s*[\r\n]+\s*Unit:\s*([^\r\n]+)'
    matches = re.findall(pattern, text, flags=re.I)
    if not matches:
        return None, None
    val_str, unit_str = matches[-1]
    val = float(val_str.replace(',', ''))
    unit_l = unit_str.strip().lower()
    # normalize unit
    if 'fl' in unit_l and 'oz' in unit_l:
        unit = 'fl_ounce'
    elif 'oz' in unit_l or 'ounce' in unit_l:
        unit = 'ounce'
    elif 'lb' in unit_l or 'pound' in unit_l:
        unit = 'pound'
    elif 'count' in unit_l or unit_l in ('ct','each'):
        unit = 'count'
    else:
        unit = unit_l
    return val, unit

def extract_weight_inline(text):
    # fallback: find inline weight like '12 Ounce', '1.5 oz', '1 Pound'
    pattern = r'(\d+(?:\.\d+)?)\s*(fl oz|fl\. oz|ounce|ounces|oz|pound|pounds|lb|lbs)\b'
    m = re.search(pattern, text, flags=re.I)
    if not m:
        return None, None
    val = float(m.group(1))
    unit_tok = m.group(2).lower()
    if 'fl' in unit_tok and 'oz' in unit_tok:
        unit = 'fl_ounce'
    elif 'oz' in unit_tok or 'ounce' in unit_tok:
        unit = 'ounce'
    elif 'lb' in unit_tok or 'pound' in unit_tok:
        unit = 'pound'
    else:
        unit = unit_tok
    return val, unit

def extract_pack_count(text):
    # Try multiple common patterns; return first match
    patterns = [
        r'pack of\s*(\d+)',
        r'\b(\d+)\s*(?:pack|packs|pk|pk\.)\b',
        r'\b(\d+)\s*(?:pc|pcs)\b',
        r'\b(\d+)\s*(?:count|ct|total)\b',
        r'\((\d+)\s*(?:pack|packs|pk|pc|pcs|count)\)',
        r'\b(\d+)-pack\b',
        r'\b(\d+)\s*packets?\b',
    ]
    for pat in patterns:
        m = re.search(pat, text, flags=re.I)
        if m:
            return int(m.group(1))
    return None

def parse_numeric_features(text):
    pack = extract_pack_count(text)
    val, unit = extract_value_unit_from_value_block(text)   # prefer Value/Unit block
    if val is None:                                         # fallback to inline
        val, unit = extract_weight_inline(text)
    # return {
    #     'pack_count': pack,
    #     'value': val,
    #     'unit': unit
    # }
    return pack, val, unit

In [ ]:
_, df['total_weight'], df['weight_unit'] = zip(*df['catalog_content'].apply(parse_numeric_features))

In [ ]:
df.isnull().sum()

,0
sample_id,0
catalog_content,0
image_link,0
price,0
total_weight,477
weight_unit,477


In [ ]:
df[df['total_weight'].isnull()]

,sample_id,catalog_content,image_link,price,total_weight,weight_unit
255,238772,Item Name: DYE INK PAD Stamperia Coffee\nBulle...,https://m.media-amazon.com/images/I/81EPOspMgs...,9.49,NaN,None
671,45478,Item Name: Curious George Goes to a Chocolate ...,https://m.media-amazon.com/images/I/51vPiWvqaM...,4.79,NaN,None
788,226144,"Item Name: Peets Coffee And Tea, Coffee Dark R...",https://m.media-amazon.com/images/I/81GWcYDITm...,34.97,NaN,None
1032,48402,Item Name: Crown Prince Tuna Tongol Sprng Wtr\...,https://m.media-amazon.com/images/I/41-8j0xhfC...,3.49,NaN,None
1247,172898,Item Name: San Giorgio Enriched Macaroni Produ...,https://m.media-amazon.com/images/I/515HQ8t6uE...,1.48,NaN,None
...,...,...,...,...,...,...
74547,140816,"Item Name: NEW WAVE Bottle 5Gl, 1 EA\nBullet P...",https://m.media-amazon.com/images/I/51KSauzXcc...,23.75,NaN,None
74594,68555,Item Name: Jwellfood Completion of Cooking Mas...,https://m.media-amazon.com/images/I/51ilCOK2EP...,22.00,NaN,None
74640,91062,Item Name: Cavanagh 128758 Commun White Wafers...,https://m.media-amazon.com/images/I/51D5Q6FANi...,34.89,NaN,None
74922,236670,Item Name: Progresso Progresso Traditional Oz\...,https://m.media-amazon.com/images/I/71pIrsD2OZ...,2.18,NaN,None


In [ ]:
median = df['total_weight'].median()

In [ ]:
df.loc[df['total_weight'].isnull().index, 'total_weight'] = median

In [ ]:
df['weight_unit'].fillna("missing", inplace = True)

/tmp/ipython-input-334053440.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['weight_unit'].fillna("missing", inplace = True)


In [ ]:
df.drop(columns = ['sample_id', 'image_link'], inplace = True)

In [ ]:
!pip install transformers datasets torch scikit-learn tqdm

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
# Keep only relevant columns
df = df[["catalog_content", "total_weight", "weight_unit", "price"]].dropna()

# Log-transform price to reduce skewness
df["price_log"] = np.log1p(df["price"])

In [ ]:
# Normalize total_weight
scaler = StandardScaler()
df["total_weight_scaled"] = scaler.fit_transform(df[["total_weight"]])

# Encode weight_unit (like g, kg, oz)
le = LabelEncoder()
df["weight_unit_enc"] = le.fit_transform(df["weight_unit"])

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
import joblib
joblib.dump(scaler, "scaler.pkl")
joblib.dump(le, "le.pkl")

['le.pkl']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

max_len = 128

def tokenize_batch(texts):
    return tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
class ProductDataset(Dataset):
    def __init__(self, df):
        self.encodings = tokenize_batch(df["catalog_content"].tolist())
        self.total_weight = torch.tensor(df["total_weight_scaled"].values, dtype=torch.float32).unsqueeze(1)
        self.weight_unit = torch.tensor(df["weight_unit_enc"].values, dtype=torch.long)
        self.labels = torch.tensor(df["price_log"].values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "total_weight": self.total_weight[idx],
            "weight_unit": self.weight_unit[idx],
            "labels": self.labels[idx],
        }

train_dataset = ProductDataset(train_df)
val_dataset = ProductDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
class BertPriceRegressor(nn.Module):
    def __init__(self, bert_model_name, num_weight_units, numeric_dim=1, hidden_dim=128):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.unit_embed = nn.Embedding(num_weight_units, 4)  # 4-dim embedding for unit
        self.fc_num = nn.Linear(numeric_dim + 4, 16)         # combine numeric + unit
        self.fc1 = nn.Linear(768 + 16, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 64)
        self.fc_out = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)
        self.act = nn.ReLU()

    def forward(self, input_ids, attention_mask, total_weight, weight_unit):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embed = bert_out.last_hidden_state[:, 0, :]  # [CLS]

        unit_emb = self.unit_embed(weight_unit)
        num_features = torch.cat([total_weight, unit_emb], dim=1)
        num_out = self.act(self.fc_num(num_features))

        combined = torch.cat([cls_embed, num_out], dim=1)
        x = self.act(self.fc1(combined))
        x = self.dropout(x)
        x = self.act(self.fc2(x))
        out = self.fc_out(x)
        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertPriceRegressor("distilbert-base-uncased", num_weight_units=len(le.classes_)).to(device)

criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 0, total_steps)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].to(device)

        preds = model(**inputs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1} Train Loss: {avg_train_loss:.4f}")

Epoch 1/3: 100%|██████████| 3750/3750 [11:07<00:00,  5.62it/s]


Epoch 1 Train Loss: 0.6728


Epoch 2/3: 100%|██████████| 3750/3750 [11:10<00:00,  5.60it/s]


Epoch 2 Train Loss: 0.4326


Epoch 3/3: 100%|██████████| 3750/3750 [11:09<00:00,  5.60it/s]

Epoch 3 Train Loss: 0.3596


In [ ]:
def smape(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    return 100 * np.mean(np.abs(y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred)) / 2))

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].cpu().numpy()
        preds = model(**inputs).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)

smape_score = smape(np.array(all_labels), np.array(all_preds))
print(f"Validation SMAPE: {smape_score:.2f}%")

Validation SMAPE: 49.16%


In [ ]:
from transformers import DistilBertTokenizer

save_dir = "distilbert_price_regressor"

# Save model + tokenizer
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("Model saved successfully to:", save_dir)

AttributeError: 'BertPriceRegressor' object has no attribute 'save_pretrained'

In [ ]:
import torch
import os

save_dir = "bert_price_regressor_manual"
os.makedirs(save_dir, exist_ok=True)

# Save model weights
torch.save(model.state_dict(), os.path.join(save_dir, "pytorch_model.bin"))

# Save tokenizer
tokenizer.save_pretrained(save_dir)

print("Model and tokenizer saved successfully.")

✅ Model and tokenizer saved successfully.


In [ ]:
test_df = pd.read_csv("/content/test.csv")
test_df.shape

(75000, 3)

In [ ]:
test_df.isnull().sum()

,0
sample_id,0
catalog_content,0
image_link,0


In [ ]:
_, test_df['total_weight'], test_df['weight_unit'] = zip(*test_df['catalog_content'].apply(parse_numeric_features))

In [ ]:
test_df.isnull().sum()

,0
sample_id,0
catalog_content,0
image_link,0
total_weight,497
weight_unit,497


In [ ]:
median = test_df['total_weight'].median()
test_df.loc[test_df['total_weight'].isnull().index, 'total_weight'] = median
test_df['weight_unit'].fillna("missing", inplace = True)
test_df.drop(columns = ['sample_id', 'image_link'], inplace = True)

/tmp/ipython-input-652730335.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['weight_unit'].fillna("missing", inplace = True)


In [ ]:
test_df.isnull().sum()

,0
catalog_content,0
total_weight,0
weight_unit,0


In [ ]:
test_encodings = tokenizer(
    test_df["catalog_content"].tolist(),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

In [5]:
import torch
import numpy as np
import pandas as pd
import joblib
from transformers import AutoTokenizer, AutoModel
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [11]:
test_df = pd.read_csv("test.csv", engine='python', on_bad_lines='skip')

In [12]:
test_df.shape

(75000, 3)

In [ ]:
test_df.columns

Index(['sample_id', 'catalog_content', 'image_link'], dtype='object')

In [ ]:
test_df.isnull().sum()

,0
sample_id,0
catalog_content,0
image_link,0


In [ ]:
import re

def extract_value_unit_from_value_block(text):
    # Find all "Value: X" followed by "Unit: Y" pairs, use last occurrence
    pattern = r'Value:\s*([0-9,]+(?:\.\d+)?)\s*[\r\n]+\s*Unit:\s*([^\r\n]+)'
    matches = re.findall(pattern, text, flags=re.I)
    if not matches:
        return None, None
    val_str, unit_str = matches[-1]
    val = float(val_str.replace(',', ''))
    unit_l = unit_str.strip().lower()
    # normalize unit
    if 'fl' in unit_l and 'oz' in unit_l:
        unit = 'fl_ounce'
    elif 'oz' in unit_l or 'ounce' in unit_l:
        unit = 'ounce'
    elif 'lb' in unit_l or 'pound' in unit_l:
        unit = 'pound'
    elif 'count' in unit_l or unit_l in ('ct','each'):
        unit = 'count'
    else:
        unit = unit_l
    return val, unit

def extract_weight_inline(text):
    # fallback: find inline weight like '12 Ounce', '1.5 oz', '1 Pound'
    pattern = r'(\d+(?:\.\d+)?)\s*(fl oz|fl\. oz|ounce|ounces|oz|pound|pounds|lb|lbs)\b'
    m = re.search(pattern, text, flags=re.I)
    if not m:
        return None, None
    val = float(m.group(1))
    unit_tok = m.group(2).lower()
    if 'fl' in unit_tok and 'oz' in unit_tok:
        unit = 'fl_ounce'
    elif 'oz' in unit_tok or 'ounce' in unit_tok:
        unit = 'ounce'
    elif 'lb' in unit_tok or 'pound' in unit_tok:
        unit = 'pound'
    else:
        unit = unit_tok
    return val, unit

def extract_pack_count(text):
    # Try multiple common patterns; return first match
    patterns = [
        r'pack of\s*(\d+)',
        r'\b(\d+)\s*(?:pack|packs|pk|pk\.)\b',
        r'\b(\d+)\s*(?:pc|pcs)\b',
        r'\b(\d+)\s*(?:count|ct|total)\b',
        r'\((\d+)\s*(?:pack|packs|pk|pc|pcs|count)\)',
        r'\b(\d+)-pack\b',
        r'\b(\d+)\s*packets?\b',
    ]
    for pat in patterns:
        m = re.search(pat, text, flags=re.I)
        if m:
            return int(m.group(1))
    return None

def parse_numeric_features(text):
    pack = extract_pack_count(text)
    val, unit = extract_value_unit_from_value_block(text)   # prefer Value/Unit block
    if val is None:                                         # fallback to inline
        val, unit = extract_weight_inline(text)
    # return {
    #     'pack_count': pack,
    #     'value': val,
    #     'unit': unit
    # }
    return pack, val, unit

In [ ]:
_, test_df['total_weight'], test_df['weight_unit'] = zip(*test_df['catalog_content'].apply(parse_numeric_features))

In [ ]:
test_df.isnull().sum()

,0
sample_id,0
catalog_content,0
image_link,0
total_weight,497
weight_unit,497


In [ ]:
median = test_df['total_weight'].median()

In [ ]:
test_df.loc[test_df['total_weight'].isnull().index, 'total_weight'] = median

In [ ]:
test_df['weight_unit'].fillna("missing", inplace = True)

/tmp/ipython-input-2996274391.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['weight_unit'].fillna("missing", inplace = True)


In [ ]:
test_df.isnull().sum()

,0
sample_id,0
catalog_content,0
image_link,0
total_weight,0
weight_unit,0


In [ ]:
test_copy = test_df.copy()

In [ ]:
test_df.drop(columns = ['sample_id', 'image_link'], inplace = True)

In [ ]:
test_df.columns

Index(['catalog_content', 'total_weight', 'weight_unit'], dtype='object')

In [ ]:
scaler = joblib.load("scaler.pkl")  # StandardScaler used on training
le     = joblib.load("le.pkl")      # LabelEncoder used on training

In [ ]:
# Normalize total_weight and encode weight_unit
test_df["total_weight_scaled"] = scaler.transform(test_df[["total_weight"]])

In [ ]:
test_df["weight_unit"] = test_df["weight_unit"].apply(lambda x: x if x in le.classes_ else 'missing')
test_df["weight_unit_enc"] = le.transform(test_df["weight_unit"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert_price_regressor_manual")

In [ ]:
max_len = 128
def tokenize_batch(texts):
    return tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.encodings = tokenize_batch(df["catalog_content"].tolist())
        self.total_weight = torch.tensor(df["total_weight_scaled"].values, dtype=torch.float32).unsqueeze(1)
        self.weight_unit = torch.tensor(df["weight_unit_enc"].values, dtype=torch.long)

    def __len__(self):
        return len(self.total_weight)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "total_weight": self.total_weight[idx],
            "weight_unit": self.weight_unit[idx]
        }

test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class BertPriceRegressor(nn.Module):
    def __init__(self, bert_model_name, num_weight_units, numeric_dim=1, hidden_dim=128):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.unit_embed = nn.Embedding(num_weight_units, 4)
        self.fc_num = nn.Linear(numeric_dim + 4, 16)
        self.fc1 = nn.Linear(768 + 16, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 64)
        self.fc_out = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)
        self.act = nn.ReLU()

    def forward(self, input_ids, attention_mask, total_weight, weight_unit):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embed = bert_out.last_hidden_state[:, 0, :]  # [CLS]

        unit_emb = self.unit_embed(weight_unit)
        num_features = torch.cat([total_weight, unit_emb], dim=1)
        num_out = self.act(self.fc_num(num_features))

        combined = torch.cat([cls_embed, num_out], dim=1)
        x = self.act(self.fc1(combined))
        x = self.dropout(x)
        x = self.act(self.fc2(x))
        out = self.fc_out(x)
        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertPriceRegressor("distilbert-base-uncased", num_weight_units=len(le.classes_)).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# Load weights
model.load_state_dict(torch.load("bert_price_regressor_manual/pytorch_model.bin", map_location=device))
model.eval()

BertPriceRegressor(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Lin

In [ ]:
all_preds = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        preds_log = model(**inputs)
        preds = torch.expm1(preds_log).cpu().numpy()  # reverse log1p transformation
        all_preds.extend(preds)

In [ ]:
test_df["predicted_price"] = np.array(all_preds).squeeze()
test_df.to_csv("test_predictions.csv", index=False)

In [ ]:
test_copy['predicted_price'] = test_df['predicted_price'].copy()

In [ ]:
test_copy.sample(5)

,sample_id,catalog_content,image_link,total_weight,weight_unit,predicted_price
5877,192684,Item Name: Mary Jane Nostalgic Candy Jar\nBull...,https://m.media-amazon.com/images/I/71EdxfPZU8...,16.0,ounce,18.664398
4753,286947,Item Name: Frontier Bulk Cranberry Orange Flav...,https://m.media-amazon.com/images/I/717pSFDWNy...,16.0,ounce,39.388168
68124,131215,"Item Name: Gatorade Thirst Quencher Powder, Le...",https://m.media-amazon.com/images/I/811k+srvqs...,16.0,ounce,24.611191
3569,26243,"Item Name: Satin Ice Fondant, Yellow, 5 Pound\...",https://m.media-amazon.com/images/I/71TZGXSaIn...,16.0,ounce,45.245598
37180,915,Item Name: Hypothermias Shaved Ice & Snow Cone...,https://m.media-amazon.com/images/I/71mQ7sQhLf...,16.0,fl_ounce,14.186828


In [ ]:
final_df = test_copy[['sample_id', 'predicted_price']].copy()

In [ ]:
final_df.to_csv("final_output.csv", index = False)